<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

# **SQL Server Machine Learning Platform**

<p align="center">
<img width="884" alt="image" src="https://user-images.githubusercontent.com/22312581/68362765-44c8bc80-00dd-11ea-9a6d-431fdb2ca2a8.png" align="center">

### <span style="color:green"><font size="2"> What will you learn today?

* How to get started on Machine Learning?
    * Where does our [documentation](https://docs.microsoft.com/en-us/sql/advanced-analytics/?view=sql-server-ver15) live?
    * Machine Learning Extension in Azure Data Studio and the concept of Dataspaces. <span style="color:blue"><font size="1"> (Work in progress)</font>
* What datset will we use?
* Building a Machine Learning Model with local Python.
* Building a Machine Learning Model using Python leveraging Spark in the SQL Server 2019 Big Data Cluster.
* Building a Machine Learning Model using R in leveraging Spark in the SQL Server 2019 Big Data Cluster.
* How are we leveraging hardware acceleration for Machine Learning specific workloads?
* How we do store models in enterprises?
    * What is MlFlow ?
    * Our contributions and commitment to MlFlow.
* How do we load the data to SQL Server using Spark?
* How do we load the data to SQL Server using Streamsets?
* How do we install Machine Learning packages in SQL Server ?
* How do we do predict on the stored data using the model in SQL Server?
* How do we do predict on the stored data using the model in Azure SQL Database?
* What is ONNX?
* How do we convert the model to ONNX?
* How do we natively score on Azure SQL Database Edge?
* What is AppDeploy?
* How do we now run the model in the container in SQL Server 2019 Big Data Cluster?
* How do we convert the model to be running as a web app in SQL Server 2019 Big Data Cluster ?

# **Let's take a look at the dataset**

## Boston Housing Dataset
Housing data contains 506 census tracts of Boston from the 1970 census.The data has following features, *medv* being the target variable:

* crim - per capita crime rate by town
* zn - proportion of residential land zoned for lots over 25,000 sq.ft
* indus - proportion of non-retail business acres per town
* chas - Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* nox - nitric oxides concentration (parts per 10 million)
* rm - average number of rooms per dwelling
* age - proportion of owner-occupied units built prior to 1940
* dis - weighted distances to five Boston employment centres
* rad - index of accessibility to radial highways
* tax - full-value property-tax rate per USD 10,000
* ptratio- pupil-teacher ratio by town
* b 1000(B - 0.63)^2, where B is the proportion of blacks by town
* lstat - percentage of lower status of the population
* medv - median value of owner-occupied homes in USD 1000’s

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

# **Building a Machine Learning Model with Local Python**
### <span style="color:red"><font size="3"> Not recommended approach for Enterprises.


In [2]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
from sklearn.datasets import load_boston
boston = load_boston()
boston

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

## Prepare the data

In [3]:
df = pd.DataFrame(data=np.c_[boston['data'], boston['target']], columns=boston['feature_names'].tolist() + ['MEDV'])

# x contains all predictors (features)
x = df.drop(['MEDV'], axis = 1)

# y is what we are trying to predict - the median value
y = df.iloc[:,-1]


 # Split the data frame into features and target
x_train = df.drop(['MEDV'], axis = 1)
y_train = df.iloc[:,-1]

print("\n*** Training data set x\n")
print(x_train.head())

print("\n*** Training data set y\n")
print(y_train.head())


*** Training data set x

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  

*** Training data set y

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64


## Building a Linear Regression Model

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('continuous',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   RobustScaler(copy=True,
                                                                                quantile_range=(25.0,
                                                                                                75.0),
                                                                                with_centering=True,
                                                                                with_scaling=True))],
                                                 

## Let's predict !

In [5]:
y_pred = model.predict(x_train)

## Score the model

In [6]:
from sklearn.metrics import r2_score, mean_squared_error
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.7406426641094094
*** Scikit-learn MSE: 21.894831181729206


## Save the model

In [7]:
from joblib import dump, load
dump(model, 'filename.joblib')
model = load('filename.joblib')
model.predict(x_train)

<p align="center">
<img src ="https://raw.githubusercontent.com/hfleitas/GlobalAIBootcamp2019/master/joblibzoom.jpg? sanitize=true" width="250" align="center">
</p>

## Predict for each feature set

In [12]:
predictions = []
for i in range(5):
    pdata = x_train.iloc[i:(i+1),:]
    p = model.predict(pdata)
    print("tuples")
    print(pdata)
    print(f"predicted: {p}")
    predictions.append(p)

print(predictions)

tuples
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE   DIS  RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.09  1.0  296.0     15.3   

       B  LSTAT  
0  396.9   4.98  
predicted: [30.00384338]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
1  0.02731  0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0     17.8   

       B  LSTAT  
1  396.9   9.14  
predicted: [25.02556238]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
2  0.02729  0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0     17.8   

        B  LSTAT  
2  392.83   4.03  
predicted: [30.56759672]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
3  0.03237  0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0     18.7   

        B  LSTAT  
3  394.63   2.94  
predicted: [28.60703649]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TA

<p align="center">
<img src ="https://miro.medium.com/max/1200/1*nPcdyVwgcuEZiEZiRqApug.jpeg" width="250" align="center">
</p>
<span style="color:blue"><font size="3"> Convert the Kernel to PySpark

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Building Machine Learning Model in Python using Spark in SQL Server 2019 Big Data Cluster**
### <span style="color:green"><font size="3"> Recommended approach for Enterprises.

In [8]:
import sklearn
import numpy as np
import pandas as pd

import sklearn.datasets
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
14,application_1572670769653_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Example: Load libraries into the cluster.

In [9]:
import subprocess

# Install JobLib
stdout = subprocess.check_output(
    "pip3 install joblib",
    stderr=subprocess.STDOUT,
    shell=True).decode("utf-8")
print(stdout)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You should consider upgrading via the 'pip install --upgrade pip' command.

## Load the data from HDFS

In [10]:
house_df = (spark.read.option("inferSchema", "true")
.option("header", "true")
.csv("/BostonData/BostonHousing.csv"))
house_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## Prepare data for Machine Learning

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

# Convert the Spark data frame to pandas
df = house_df.toPandas()

x_train = df.drop(['medv'], axis = 1)
y_train = df.iloc[:,-1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Build a Linear Regression Model

In [12]:
continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
lr_model = model.fit(x_train, y_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Let's Predict !

In [13]:
y_pred = model.predict(x_train)
print(y_pred)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[30.00384338 25.02556238 30.56759672 28.60703649 27.94352423 25.25628446
 23.00180827 19.53598843 11.52363685 18.92026211 18.99949651 21.58679568
 20.90652153 19.55290281 19.28348205 19.29748321 20.52750979 16.91140135
 16.17801106 18.40613603 12.52385753 17.67103669 15.83288129 13.80628535
 15.67833832 13.38668561 15.46397655 14.70847428 19.54737285 20.8764282
 11.45511759 18.05923295  8.81105736 14.28275814 13.70675891 23.81463526
 22.34193708 23.10891142 22.91502612 31.35762569 34.21510225 28.02056414
 25.20386628 24.60979273 22.94149176 22.09669817 20.42320032 18.03655088
  9.10655377 17.20607751 21.28152535 23.97222285 27.6558508  24.04901809
 15.3618477  31.15264947 24.85686978 33.10919806 21.77537987 21.08493555
 17.8725804  18.51110208 23.98742856 22.55408869 23.37308644 30.36148358
 25.53056512 21.11338564 17.42153786 20.78483633 25.20148859 21.7426577
 24.55744957 24.04295712 25.50499716 23.9669302  22.94545403 23.35699818
 21.26198266 22.42817373 28.40576968 26.99486086 26.0

## Score the Model

In [14]:
from sklearn.metrics import r2_score, mean_squared_error
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

*** Scikit-learn r2 score: 0.7406426641094094
*** Scikit-learn MSE: 21.894831181729206

## Save the model in HDFS

In [15]:
from joblib import dump, load
dump(model, 'filename.joblib')
import subprocess
cmd='/opt/hadoop/bin/hadoop fs -copyFromLocal -f filename.joblib /user/rony/'
subprocess.check_output(cmd.split())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b''

<p align="center">
<img src ="https://upload.wikimedia.org/wikipedia/en/thumb/2/29/Apache_Spark_Logo.svg/1200px-Apache_Spark_Logo.svg.png" width="250" align="center">
</p>

<span style="color:blue"><font size="3"> Convert the Kernel to Spark | R in the Kernel

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Building Machine Learning Model in R using Spark in SQL Server 2019 Big Data Cluster**
### <span style="color:green"><font size="3"> Recommended approach for Enterprises.

## Install packages in the cluster

In [5]:
install.packages('caTools')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1572670769653_0016,sparkr,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Installing package into '/tmp/nm-local-dir/filecache/10/sparkr.zip'
(as 'lib' is unspecified)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63565  100 63565    0     0  61777      0  0:00:01  0:00:01 --:--:-- 61833
* installing *source* package 'caTools' ...
** package 'caTools' successfully unpacked and MD5 sums checked
** libs
g++  -I/opt/microsoft/ropen/3.5.2/lib64/R/include -DNDEBUG   -DU_STATIC_IMPLEMENTATION   -fpic  -DU_STATIC_IMPLEMENTATION -O2 -g  -c Gif2R.cpp -o Gif2R.o
g++  -I/opt/microsoft/ropen/3.5.2/lib64/R/include -DNDEBUG   -DU_STATIC_IMPLEMENTATION   -fpic  -DU_STATIC_IMPLEMENTATION -O2 -g  -c GifTools.cpp -o GifTools.o
gcc -std=gnu99 -I/opt/microsoft/ropen/3.5.2/lib64/R/include -DNDEBUG   -DU_STATIC_IMPLEMENTATION   -fpic  -DU_STATIC_IMPLEMENTATION -O2 -g  -c runfunc.c -o runfunc.o
g++ -shared -L/opt/microsoft/ropen/3.5.2/lib64/R/lib -o caTools.so Gif2R.

## Load the library.

In [6]:
library(caTools)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load the Boston Housing dataset

In [7]:
library(MASS)
housing <- Boston
str(housing)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Attaching package: 'MASS'

The following object is masked from 'package:SparkR':

    select

'data.frame':	506 obs. of  14 variables:
 $ crim   : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ zn     : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ indus  : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ chas   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ nox    : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ rm     : num  6.58 6.42 7.18 7 7.15 ...
 $ age    : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ dis    : num  4.09 4.97 4.97 6.06 6.06 ...
 $ rad    : int  1 2 2 3 3 3 5 5 5 5 ...
 $ tax    : num  296 242 242 222 222 222 311 311 311 311 ...
 $ ptratio: num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ black  : num  397 397 393 395 397 ...
 $ lstat  : num  4.98 9.14 4.03 2.94 5.33 ...
 $ medv   : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...

## Load summary of the data

In [8]:
summary(housing)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      crim                zn             indus            chas        
 Min.   : 0.00632   Min.   :  0.00   Min.   : 0.46   Min.   :0.00000  
 1st Qu.: 0.08204   1st Qu.:  0.00   1st Qu.: 5.19   1st Qu.:0.00000  
 Median : 0.25651   Median :  0.00   Median : 9.69   Median :0.00000  
 Mean   : 3.61352   Mean   : 11.36   Mean   :11.14   Mean   :0.06917  
 3rd Qu.: 3.67708   3rd Qu.: 12.50   3rd Qu.:18.10   3rd Qu.:0.00000  
 Max.   :88.97620   Max.   :100.00   Max.   :27.74   Max.   :1.00000  
      nox               rm             age              dis        
 Min.   :0.3850   Min.   :3.561   Min.   :  2.90   Min.   : 1.130  
 1st Qu.:0.4490   1st Qu.:5.886   1st Qu.: 45.02   1st Qu.: 2.100  
 Median :0.5380   Median :6.208   Median : 77.50   Median : 3.207  
 Mean   :0.5547   Mean   :6.285   Mean   : 68.57   Mean   : 3.795  
 3rd Qu.:0.6240   3rd Qu.:6.623   3rd Qu.: 94.08   3rd Qu.: 5.188  
 Max.   :0.8710   Max.   :8.780   Max.   :100.00   Max.   :12.127  
      rad              tax 

## Train and Test Data
Lets split the data into train and test data using caTools library.

In [9]:
set.seed(123)

split <- sample.split(housing,SplitRatio =0.75)

train <- subset(housing,split==TRUE)
test <- subset(housing,split==FALSE)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Training our Model
Lets build our model considering that crim,rm,tax,lstat as the major influencers on the target variable.

### General Form
The General Linear regression model in R :

Univariate Model : model<−lm(y∼x,data)

Multivariate Model : model<−lm(y∼.,data)

In [10]:
model <- lm(medv ~ crim + rm + tax + lstat, data = train)
summary(model)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Call:
lm(formula = medv ~ crim + rm + tax + lstat, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-16.266  -3.185  -1.052   2.116  30.121 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.767079   3.573477  -1.054  0.29251    
crim        -0.070793   0.037113  -1.908  0.05725 .  
rm           5.580390   0.492854  11.323  < 2e-16 ***
tax         -0.006392   0.002114  -3.023  0.00268 ** 
lstat       -0.483836   0.058230  -8.309 2.04e-15 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.183 on 357 degrees of freedom
Multiple R-squared:  0.6816,	Adjusted R-squared:  0.678 
F-statistic: 191.1 on 4 and 357 DF,  p-value: < 2.2e-16


## Predictions
Let’s test our model by predicting on our testing dataset.

In [11]:
test$predicted.medv <- predict(model,test)
test$predicted.medv

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  [1] 32.440740 32.113253 19.411543 19.920849 20.543782 19.004375 20.805892
  [8] 19.389226 21.749638 13.968600 22.869986 22.828563 21.499305 20.172152
 [15] 18.151272 28.455183 23.024340 20.081653 27.546848 21.415019 25.471001
 [22] 25.408773 22.977419 29.413703 24.723074 30.943821 26.279292 24.252127
 [29] 27.899978 24.361495 18.451163 23.960183 18.914829 22.101451 18.857622
 [36] 19.614202 15.960423 23.357454 18.633330 18.398711 11.097580  6.649332
 [43]  6.690665 20.880808 30.094244 24.301541 34.507933 20.534033 20.485362
 [50] 18.317478 28.286616 29.269575 22.968716 36.542299 31.170285 32.350281
 [57] 30.378115 31.325170 36.778058 24.439996 25.489047 10.345608 26.840397
 [64] 28.336530 31.136580 35.243768 33.059574 27.877751 22.320777 24.359574
 [71] 16.456553 25.404518 22.531880 34.763728 29.341558 38.513306 21.232677
 [78] 21.175265 34.526978 32.227477 37.650788 30.166307 25.551434 31.340331
 [85] 19.022201 26.942988 26.305841 31.836754 25.503665 22.211417 20.543873
 [92] 24.862

## Evaluating the model

In [12]:
error <- test$medv-test$predicted.medv
rmse <- sqrt(mean(error)^2)
rmse

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1] 0.7602882

<p align="center">
<img src ="https://miro.medium.com/max/1200/1*nPcdyVwgcuEZiEZiRqApug.jpeg" width="200" align="center">
</p>
<span style="color:blue"><font size="3"> Convert the Kernel to PySpark

## **Load the data into SQL using the Spark to SQL Connector**

In [7]:
house_df = (spark.read.option("inferSchema", "true")
.option("header", "true")
.csv("/BostonData/BostonHousing.csv"))
house_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [8]:
#Write from Spark to SQL table using MSSQL Spark Connector
import os

# Read credentials for server
servername = "jdbc:sqlserver://master-0.master-svc"
df = (spark.read.option("inferSchema", "true")
.option("header", "true")
.csv("/user/rony/credentials.txt"))

credentials = df.filter(df.servername == servername).collect()

print("Use MSSQL connector to write to master SQL instance ")
#servername="jdbc:sqlserver://10.193.17.192:31433"
dbname = "BostonData"
url = servername + ";" + "databaseName=" + dbname + ";"
datasource_name = "Boston"
dbtable = "Boston"
user = credentials[0].username
password = credentials[0].password
#com.microsoft.sqlserver.jdbc.spark
try:
  house_df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .save()
    
except ValueError as error :
    print("MSSQL Connector write failed", error)
print("MSSQL Connector write(overwrite) succeeded  ")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use MSSQL connector to write to master SQL instance 
MSSQL Connector write(overwrite) succeeded


# **Partner Engagements**

## **Load the data into SQL Server Big Data Clusters from Kafka using StreamSets**

<img width="166" alt="image" src="https://user-images.githubusercontent.com/22312581/68363893-2b297400-00e1-11ea-980b-1abf3c790269.png">


### StreamSets Demo

## **Accelerating Spark ML over SQL Server 2019 Big Data Cluster**

<img width="127" alt="image" src="https://user-images.githubusercontent.com/22312581/68370687-ec4fea00-00f1-11ea-87ca-a08f37cb0fbd.png">

[InAccel Blog Post](https://medium.com/@inaccel/accelerating-spark-ml-over-sql-server-2019-big-data-cluster-instantly-542159267052)

<img width="707" alt="image" src="https://user-images.githubusercontent.com/22312581/68371038-a5aebf80-00f2-11ea-8cc7-9b4dfdc76c7f.png">


## **Package Management in SQL Server**

In [1]:
EXEC sp_execute_external_script
@language=N'Python',
@script=N'import pkg_resources
import pandas
OutputDataSet = pandas.DataFrame([(d.project_name, d.version) for d in pkg_resources.working_set])
'

(169 rows affected)

Total execution time: 00:00:03.692

(No column name),(No column name)
glue,0.13
revoscalepy,9.4.7
microsoftml,9.4.7
zipp,0.5.2
zict,1.0.0
xlwt,1.3.0
XlsxWriter,1.1.2
xlrd,1.2.0
wrapt,1.11.2
widgetsnbextension,3.4.2


<p align="center">
<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/1024px-Python-logo-notext.svg.png" width="100" align="center">
</p>

<span style="color:blue"><font size="3"> Convert the Kernel to Python 3

Get credentials for each server from credentials.txt.

In [27]:
import sqlmlutils

# For Linux SQL Server, you must specify the ODBC Driver and the username/password because there is no Trusted_Connection/Implied Authentication support yet.

import os
credentials_dict = {}
cwd = os.getcwd()
with open('/Users/submarine/Downloads/IgniteML/MachineLearning/credentials.txt') as f:
    for line in f:
        login_information = line.split(',')
        key = login_information[0]
        value = [login_information[1], login_information[2]]
        credentials_dict[key] = value

servername = "13.82.197.135:31433"
username = credentials_dict[servername][0]
password = credentials_dict[servername][1]
server = "13.82.197.135"

connection = sqlmlutils.ConnectionInfo(driver="ODBC Driver 17 for SQL Server", server=server, port=1433, uid=username, pwd=password, database="ADUG")
sqlpy = sqlmlutils.SQLPythonExecutor(connection)
pkgmanager = sqlmlutils.SQLPackageManager(connection)
pkgmanager.install("glue", upgrade=True)

In [6]:
pkgmanager.uninstall("jinja2")

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Model Management in SQL Server**
### <span style="color:green"><font size="3"> Recommended approach for Enterprises.

## What is MlFlow?
What is MlFlow?
MlFlow is an open-source model management system.

* Create experiments
* Track machine learning models, parameters, and metrics
* Visualize the results

We have enhanced MlFlow to log machine learning models into SQL Server and exploit SQL Server's data governance, security and model management features.

<p align="center">
<img src ="https://miro.medium.com/max/1200/1*nPcdyVwgcuEZiEZiRqApug.jpeg" width="200" align="center">
</p>
<span style="color:blue"><font size="3"> Convert the Kernel to PySpark

## Import MlFlow

In [9]:
import mlflow
import mlflow.sklearn
import os

# Read credentials for server
servername = "13.82.197.135:31433"
df = (spark.read.option("inferSchema", "true")
.option("header", "true")
.csv("/user/rony/credentials.txt"))

credentials = df.filter(df.servername == servername).collect()
username = credentials[0].username
password = credentials[0].password

mlflow.set_tracking_uri('http://40.85.166.63:5000/')
db_uri_artifact = "mssql+pyodbc://" + username + ":" + password + "@" + "server"+"/MlFlow?driver=ODBC+Driver+17+for+SQL+Server"

#create new experiment
exp_name = "artifact_test_experiment_04"

mlflow.create_experiment(exp_name, artifact_location=db_uri_artifact)
mlflow.set_experiment(exp_name)

## Build a Model wrapped with MlFlow

In [11]:
import subprocess

# Install PyODBC
stdout = subprocess.check_output(
    "pip3 install pyodbc",
    stderr=subprocess.STDOUT,
    shell=True).decode("utf-8")
print(stdout)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for pyodbc: filename=pyodbc-4.0.27-cp35-cp35m-linux_x86_64.whl size=213950 sha256=99e65007db45beda0cbf4efb46eebe04426d37b520b53dddcd4c7c5ecccb163f
  Stored in directory: /home/.cache/pip/wheels/87/b7/78/0740a2ed98bfe463525ad42d535370e34141c5d36b2d00dcaf
Successfully built pyodbc
You should consider upgrading via the 'pip install --upgrade pip' command.

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

house_df = (spark.read.option("inferSchema", "true")
.option("header", "true")
.csv("/BostonData/BostonHousing.csv"))

# Convert the Spark data frame to pandas
df = house_df.toPandas()

x_train = df.drop(['medv'], axis = 1)
y_train = df.iloc[:,-1]

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
lr_model = model.fit(x_train, y_train)

with mlflow.start_run(nested=True):
    name = model.__class__.__name__
    mlflow.sklearn.log_model(lr_model,name)

<p align="center">
<img src ="https://sezeromer.com/wp-content/uploads/2018/01/sql.png" width="200" align="center">
</p>
<span style="color:blue"><font size="3"> Convert the Kernel to SQL


## Viewing the models in SQL Server as the backend datastore for MlFlow

In [1]:
SELECT * FROM [MlFlow].[dbo].[artifacts]

(6 rows affected)

Total execution time: 00:00:00.037

artifact_id,artifact_name,group_path,artifact_content,artifact_initial_size
1,MLmodel,c3e17c05ece14749bb6ae355f1060a21/artifacts/Pipeline,0x61727469666163745F706174683A20506970656C696E650A666C61766F72733A0A2020707974686F6E5F66756E6374696F6E3A0A20202020646174613A206D6F64656C2E706B6C0A20202020656E763A20636F6E64612E79616D6C0A202020206C6F616465725F6D6F64756C653A206D6C666C6F772E736B6C6561726E0A20202020707974686F6E5F76657273696F6E3A20332E352E320A2020736B6C6561726E3A0A202020207069636B6C65645F6D6F64656C3A206D6F64656C2E706B6C0A2020202073657269616C697A6174696F6E5F666F726D61743A20636C6F75647069636B6C650A20202020736B6C6561726E5F76657273696F6E3A20302E32312E330A72756E5F69643A2063336531376330356563653134373439626236616533353566313036306132310A7574635F74696D655F637265617465643A2027323031392D31312D30352031393A34313A31332E383533313439270A,346
2,conda.yaml,c3e17c05ece14749bb6ae355f1060a21/artifacts/Pipeline,0x6368616E6E656C733A0A2D2064656661756C74730A646570656E64656E636965733A0A2D20707974686F6E3D332E352E320A2D207363696B69742D6C6561726E3D302E32312E330A2D207069703A0A20202D206D6C666C6F770A20202D20636C6F75647069636B6C653D3D312E322E310A6E616D653A206D6C666C6F772D656E760A,130
3,model.pkl,c3e17c05ece14749bb6ae355f1060a21/artifacts/Pipeline,0x80049521060000000000008C10736B6C6561726E2E706970656C696E65948C08506970656C696E659493942981947D94288C07766572626F736594898C105F736B6C6561726E5F76657273696F6E948C06302E32312E33948C057374657073945D94288C0C70726570726F636573736F72948C23736B6C6561726E2E636F6D706F73652E5F636F6C756D6E5F7472616E73666F726D6572948C11436F6C756D6E5472616E73666F726D65729493942981947D9428680668078C066E5F6A6F6273944E8C0A5F72656D61696E646572948C0972656D61696E646572948C0464726F70944E87946805898C137472616E73666F726D65725F77656967687473944E8C085F636F6C756D6E73945D945D94284B004B014B024B034B044B054B064B074B084B094B0A4B0B4B0C65618C0C7472616E73666F726D657273945D948C0A636F6E74696E756F75739468022981947D94286805896806680768085D948C067363616C6572948C1A736B6C6561726E2E70726570726F63657373696E672E64617461948C0C526F627573745363616C65729493942981947D9428680668078C04636F707994888C0C776974685F7363616C696E6794888C0E7175616E74696C655F72616E676594474039000000000000474052C0000000000086948C0E776974685F63656E746572696E67948875628694618C066D656D6F7279944E756268188794618C0E7370617273655F6F75747075745F94898C107370617273655F7468726573686F6C6494473FD33333333333338C0D7472616E73666F726D6572735F945D94681B68022981947D94286805896806680768085D94681F68222981947D94288C0763656E7465725F948C156E756D70792E636F72652E6D756C74696172726179948C0C5F7265636F6E7374727563749493948C056E756D7079948C076E6461727261799493944B0085944301629487945294284B014B0D85948C056E756D7079948C0564747970659493948C026638944B004B0187945294284B038C013C944E4E4E4AFFFFFFFF4AFFFFFFFF4B00749462894368983446EBA86AD03F0000000000000000E17A14AE4761234000000000000000009EEFA7C64B37E13F2FDD240681D5184000000000006053400C71AC8BDBA8094000000000000014400000000000A07440CDCCCCCCCC0C3340D7A3703D0A777840B81E85EB51B8264094749462680668076825886827474039000000000000474052C0000000000086946829888C067363616C655F946839683C4B008594683E87945294284B014B0D8594684789436818265305A3C20C40000000000000294052B81E85EBD12940000000000000F03F666666666666C63F00560E2DB29DE73F6566666666864840DCF97E6ABCB40840000000000000344000000000003078406866666666660640305C8FC2F5D83440C4F5285C8F022440947494626826887562869461682B4E756268188794618C0B5F6E5F6665617475726573944B0D68126813756286948C09726567726573736F72948C19736B6C6561726E2E6C696E6561725F6D6F64656C2E62617365948C104C696E65617252656772657373696F6E9493942981947D9428680668078C05636F65665F946839683C4B008594683E87945294284B014B0D8594684789436879D81AB8FCD9D8BF1E6FBC767491E23F7AD4890D82FCD03FD925F94C6E7E054067D0D4AF8DDF08C0ABCD9B5B527E0640782103635E62A13F17BC4B0E493A12C0D1EE3FB1E47B18402BC9A4320D1813C0C96FA2086F5705C08051073019D9C83FCF13D169360015C0947494628C06636F70795F5894888C0572616E6B5F944B0D8C096E6F726D616C697A6594898C0A696E746572636570745F9468378C067363616C6172949394684743084C462F4E680A3740948694529468104E8C0D6669745F696

## **Deploy the models to SQL Server using MlFlow**

In [0]:
import mlflow.database
db_uri_deployment = "mssql+pyodbc://igniteuser@10.193.17.192:1433/modelstore?driver=ODBC+Driver+17+for+SQL+Server"
mlflow.database.deploy(model_uri=model_uri, db_uri=db_uri_deployment, flavor='onnx', table_name=None, column_name=None)
print("Deployed the model to Azure SQL Database.")

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Scoring the models in SQL Server**
### <span style="color:green"><font size="3"> Recommended approach for Enterprises.

## **In Azure SQL Database using R**

In [1]:
select len(data), description from models

(1 row affected)

Total execution time: 00:00:00.038

(No column name),description
76402,R Model


In [2]:
declare @model varbinary(max) = (select [data] from models where description = 'R Model')
exec sp_execute_external_script
    @language = N'R',
    @script = N'
        rmodel <- unserialize(model);
        OutputDataset <- data.frame(MEDV=predict(rmodel, InputDataset))
    ',
    @input_data_1 = N'select top(10)* from features',
	@input_data_1_name = N'InputDataset',
    @output_data_1_name = N'OutputDataset',
    @params = N'@model varbinary(max)',  
    @model = @model
    WITH RESULT SETS ((MEDV float)); 

STDOUT message(s) from external script: 
MKL_VERBOSE Intel(R) MKL 2017.0 Product build 20160801 for Intel(R) 64 architecture Intel(R) Advanced Vector Extensions 2 (Intel(R) AVX2) enabled processors, Win 2.30GHz cdecl intel_thread NMICDev:0
MKL_VERBOSE DGEMV(N,28,4,0000000704225A70,0000000712CF26E8,28,0000000712836998,1,0000000704225A80,00000007122016C0,1) 146.25us CNR:AUTO Dyn:1 FastMM:1 TID:0 NThr:4 WDiv:HOST:+0.000
MKL_VERBOSE DGEMV(N,27,4,0000000704225A70,0000000712A68580,27,0000000712DDFE18,1,0000000704225A80,00000007127BBD28,1) 16.76us CNR:AUTO Dyn:1 FastMM:1 TID:0 NThr:4 WDiv:HOST:+0.000
NULL
MKL_VERBOSE DGEMV(N,10,5,0000000704231270,00000007169D76E0,10,0000000716A14FD0,1,0000000704231280,00000007169E7AC8,1) 3.03us CNR:AUTO Dyn:1 FastMM:1 TID:0 NThr:4 WDiv:HOST:+0.000

(10 rows affected)

Total execution time: 00:00:06.778

MEDV
28.9460682828789
26.1825571200056
32.9255884539791
32.6271050128186
32.128649669109
28.4319111254368
21.9261917675329
19.1684304710369
10.5592430953262
19.3814877930924


## **In SQL Server Big Data Cluster using Python**

In [1]:
select * from [MlFlow].dbo.artifacts where artifact_id = 3

(1 row affected)

Total execution time: 00:00:00.039

artifact_id,artifact_name,group_path,artifact_content,artifact_initial_size
3,model.pkl,c3e17c05ece14749bb6ae355f1060a21/artifacts/Pipeline,0x80049521060000000000008C10736B6C6561726E2E706970656C696E65948C08506970656C696E659493942981947D94288C07766572626F736594898C105F736B6C6561726E5F76657273696F6E948C06302E32312E33948C057374657073945D94288C0C70726570726F636573736F72948C23736B6C6561726E2E636F6D706F73652E5F636F6C756D6E5F7472616E73666F726D6572948C11436F6C756D6E5472616E73666F726D65729493942981947D9428680668078C066E5F6A6F6273944E8C0A5F72656D61696E646572948C0972656D61696E646572948C0464726F70944E87946805898C137472616E73666F726D65725F77656967687473944E8C085F636F6C756D6E73945D945D94284B004B014B024B034B044B054B064B074B084B094B0A4B0B4B0C65618C0C7472616E73666F726D657273945D948C0A636F6E74696E756F75739468022981947D94286805896806680768085D948C067363616C6572948C1A736B6C6561726E2E70726570726F63657373696E672E64617461948C0C526F627573745363616C65729493942981947D9428680668078C04636F707994888C0C776974685F7363616C696E6794888C0E7175616E74696C655F72616E676594474039000000000000474052C0000000000086948C0E776974685F63656E746572696E67948875628694618C066D656D6F7279944E756268188794618C0E7370617273655F6F75747075745F94898C107370617273655F7468726573686F6C6494473FD33333333333338C0D7472616E73666F726D6572735F945D94681B68022981947D94286805896806680768085D94681F68222981947D94288C0763656E7465725F948C156E756D70792E636F72652E6D756C74696172726179948C0C5F7265636F6E7374727563749493948C056E756D7079948C076E6461727261799493944B0085944301629487945294284B014B0D85948C056E756D7079948C0564747970659493948C026638944B004B0187945294284B038C013C944E4E4E4AFFFFFFFF4AFFFFFFFF4B00749462894368983446EBA86AD03F0000000000000000E17A14AE4761234000000000000000009EEFA7C64B37E13F2FDD240681D5184000000000006053400C71AC8BDBA8094000000000000014400000000000A07440CDCCCCCCCC0C3340D7A3703D0A777840B81E85EB51B8264094749462680668076825886827474039000000000000474052C0000000000086946829888C067363616C655F946839683C4B008594683E87945294284B014B0D8594684789436818265305A3C20C40000000000000294052B81E85EBD12940000000000000F03F666666666666C63F00560E2DB29DE73F6566666666864840DCF97E6ABCB40840000000000000344000000000003078406866666666660640305C8FC2F5D83440C4F5285C8F022440947494626826887562869461682B4E756268188794618C0B5F6E5F6665617475726573944B0D68126813756286948C09726567726573736F72948C19736B6C6561726E2E6C696E6561725F6D6F64656C2E62617365948C104C696E65617252656772657373696F6E9493942981947D9428680668078C05636F65665F946839683C4B008594683E87945294284B014B0D8594684789436879D81AB8FCD9D8BF1E6FBC767491E23F7AD4890D82FCD03FD925F94C6E7E054067D0D4AF8DDF08C0ABCD9B5B527E0640782103635E62A13F17BC4B0E493A12C0D1EE3FB1E47B18402BC9A4320D1813C0C96FA2086F5705C08051073019D9C83FCF13D169360015C0947494628C06636F70795F5894888C0572616E6B5F944B0D8C096E6F726D616C697A6594898C0A696E746572636570745F9468378C067363616C6172949394684743084C462F4E680A3740948694529468104E8C0D6669745F696E7465726365707494888C095F726573696475657394686A68474308E6E80C6D64A3C54094869452948C0973696E67756C61725F946839683C4B008594683E87945294284B014B0D859468478943689BB7866A58B7594066A4279B56FB494077D7047B8600454025EF3131F7BF3540032EBFB97F4B3140CF4DC91356C72D40499631CF06E12240287AF3B4F0761F40F47C573E11121A40D7A5B642F3161840C634401B8E2B17405E0C8A8E9F0D1640F3D8720680FF0340947494627562869465682B4E75622E,1580


In [2]:
declare @model varbinary(max) = (select artifact_content from [MlFlow].dbo.artifacts where artifact_id = 3)
exec sp_execute_external_script
    @language = N'Python',
    @script = N'
import pandas
import pickle
import sklearn

scikit_model = pickle.loads(model)
output_dataset = pandas.DataFrame(data=scikit_model.predict(input_dataset), columns=["MEDV"])
    ',
    @input_data_1 = N'select top(10)* from [BostonData].dbo.Boston',
	@input_data_1_name = N'input_dataset',
    @output_data_1_name = N'output_dataset',
    @params = N'@model varbinary(max)',  
    @model = @model
    WITH RESULT SETS ((MEDV float)); 

STDERR message(s) from external script: 
/opt/mssql/mlservices/runtime/python/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator RobustScaler from version 0.21.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)
/opt/mssql/mlservices/runtime/python/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)
/opt/mssql/mlservices/runtime/python/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.21.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)

STDERR message(s) from external script: 
/opt/mssql/mlservices/runtime/python/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
 UserWarning)

(10 rows affected)

Total execution time: 00:00:06.914

MEDV
30.0038433770168
25.0255623790531
30.5675967186016
28.6070364887281
27.943524232873
25.2562844615411
23.0018082684854
19.5359884287561
11.5236368531304
18.920262107076


<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Azure SQL Database Edge**
### <span style="color:green"><font size="3"> Recommended approach for Enterprises.

In [3]:
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
import skl2onnx
import sklearn
import sklearn.datasets

from sklearn.datasets import load_boston
boston = load_boston()

df = pd.DataFrame(data=np.c_[boston['data'], boston['target']], columns=boston['feature_names'].tolist() + ['MEDV'])

# x contains all predictors (features). in this version a single column is used (NOX)
x_train = pd.DataFrame(df.iloc[:,df.columns.tolist().index('NOX')])

# y is what we are trying to predict - the median value
y_train = pd.DataFrame(df.iloc[:,-1])

print("\n*** Training data set x\n")
print(x_train.head())

print("\n*** Training data set y\n")
print(y_train.head())


*** Training data set x

     NOX
0  0.538
1  0.469
2  0.469
3  0.458
4  0.458

*** Training data set y

   MEDV
0  24.0
1  21.6
2  34.7
3  33.4
4  36.2


In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

noxmodel = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
noxmodel.fit(x_train, y_train)

preds = []
for i in range(5):
    pdata = x_train.iloc[i:(i+1),:]
    p = noxmodel.predict(pdata)
    print("tuples")
    print(pdata)
    print("predicted:",p)
    preds.append(p)

print(preds)

tuples
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE   DIS  RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.09  1.0  296.0     15.3   

       B  LSTAT  
0  396.9   4.98  
predicted: [30.00384338]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
1  0.02731  0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0     17.8   

       B  LSTAT  
1  396.9   9.14  
predicted: [25.02556238]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
2  0.02729  0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0     17.8   

        B  LSTAT  
2  392.83   4.03  
predicted: [30.56759672]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  PTRATIO  \
3  0.03237  0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0     18.7   

        B  LSTAT  
3  394.63   2.94  
predicted: [28.60703649]
tuples
      CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TA

## **What is ONNX ?**
Using skl2onnx, we will convert our LinearRegression model to the ONNX format and save it locally.

In [5]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

In [6]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(noxmodel, 'Boston NOX Data', convert_dataframe_schema(x_train))
# Save the onnx model locally
# onnx_model_path = 'boston1.model.onnx'
# onnxmltools.utils.save_model(onnx_model, onnx_model_path)

## **Test the ONNX model**
After converting the model to ONNX format, we score the model to show little to no degradation in performance.

*ONNX Runtime uses floats instead of doubles which explains small potential discrepencies.*

In [7]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model.SerializeToString())

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = sklearn.metrics.r2_score(y_train, y_pred)
onnx_mse = sklearn.metrics.mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print()



*** Onnx r2 score: 0.18260303162826075
*** Onnx MSE: 69.00428927333752




## **Insert the ONNX model into Azure SQL Database Edge**
Now, we will store the model in SQL Server. We will create a database ```onnx``` with a ```models``` table to store the ONNX model. In the connection string, you will need to specify the **server address, username, and password**. You will need to also import the *pyodbc* package.

In [19]:
import pyodbc

import os
credentials_dict = {}

with open('/Users/submarine/Downloads/IgniteML/MachineLearning/credentials.txt') as f:
    for line in f:
        login_information = line.split(',')
        key = login_information[0]
        value = [login_information[1], login_information[2]]
        credentials_dict[key] = value

server = "10.193.17.192" # SQL Server DB Edge IP address
username = credentials_dict[server][0] # SQL Server username
password = credentials_dict[server][1] # SQL Server password

# Connect to the master DB to create the new onnx database
connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=master;UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

database = 'edgeonnxdb'
query = 'DROP DATABASE IF EXISTS ' + database
cursor.execute(query)
conn.commit()

# Create onnx database
query = 'CREATE DATABASE ' + database
cursor.execute(query)
conn.commit()

# Connect to onnx database

db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()

table_name = 'models'

# Drop the table if it exists
query = f'drop table if exists {table_name}'
cursor.execute(query)
conn.commit()

# Create the model table
query = f'create table {table_name} ( ' \
    f'[id] [int] IDENTITY(1,1) NOT NULL, ' \
    f'[data] [varbinary](max) NULL, ' \
    f'[description] varchar(1000))'
cursor.execute(query)
conn.commit()

# Insert the ONNX model into the models table
query = f"insert into {table_name} ([description], [data]) values ('Onnx Model',?)"

model_bits = onnx_model.SerializeToString()

insert_params  = (pyodbc.Binary(model_bits))
cursor.execute(query, insert_params)
conn.commit()

## **Load the data into SQL Server**

We will create two tables, **features** and **target**, to store subsets of the boston housing dataset. 
* Features will contain all data being used to predict the target, median value. 
* Target contains the median value for each record in the dataset. 

You will need to import the *sqlalchemy* package.

In [20]:
import sqlalchemy
from sqlalchemy import create_engine
import urllib

db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(db_connection_string)
cursor = conn.cursor()

features_table_name = 'features'

# Drop the table if it exists
query = f'drop table if exists {features_table_name}'
cursor.execute(query)
conn.commit()

# Create the features table
query = \
    f'create table {features_table_name} ( ' \
    f'    [CRIM] float, ' \
    f'    [ZN] float, ' \
    f'    [INDUS] float, ' \
    f'    [CHAS] float, ' \
    f'    [NOX] float, ' \
    f'    [RM] float, ' \
    f'    [AGE] float, ' \
    f'    [DIS] float, ' \
    f'    [RAD] float, ' \
    f'    [TAX] float, ' \
    f'    [PTRATIO] float, ' \
    f'    [B] float, ' \
    f'    [LSTAT] float, ' \
    f'    [id] int)'

cursor.execute(query)
conn.commit()

target_table_name = 'target'

# Create the target table
query = \
    f'create table {target_table_name} ( ' \
    f'    [MEDV] float, ' \
    f'    [id] int)'

x_train['id'] = range(1, len(x_train)+1)
y_train['id'] = range(1, len(y_train)+1)

print(x_train.head())
print(y_train.head())

     NOX  id
0  0.538   1
1  0.469   2
2  0.469   3
3  0.458   4
4  0.458   5
   MEDV  id
0  24.0   1
1  21.6   2
2  34.7   3
3  33.4   4
4  36.2   5


Finally, using sqlalchemy, we insert the `x_train` and `y_train` pandas dataframes into tables `features` and `target`, respectively. 

In [14]:
db_connection_string = 'mssql+pyodbc://' + username + ':' + password + '@' + server + '/' + database + '?driver=ODBC+Driver+17+for+SQL+Server'
sql_engine = sqlalchemy.create_engine(db_connection_string)
x_train.to_sql(features_table_name, sql_engine, if_exists='append', index=False)
y_train.to_sql(target_table_name, sql_engine, if_exists='append', index=False)

You will now be able to view the data in SQL.

<p align="center">
<img src ="https://sezeromer.com/wp-content/uploads/2018/01/sql.png" width="200" align="center">
</p>

<span style="color:blue"><font size="3"> Convert the Kernel to SQL

<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>

## **Native PREDICT on Azure SQL Database Edge**

Below, using the stored ONNX model, we predict the median value of a house based on other features from the boston housing dataset. 

In [1]:
USE onnx

DECLARE @model varbinary(max) = (SELECT DATA FROM dbo.models WHERE id = 1);
WITH predict_input as (SELECT TOP (1000) [id]
      ,CRIM
      ,ZN
      ,INDUS
      ,CHAS
      ,NOX
      ,RM
      ,AGE
      ,DIS
      ,RAD
      ,TAX
      ,PTRATIO
      ,B
      ,LSTAT
  FROM [onnx].[dbo].[features])

SELECT predict_input.id, p.variable1 AS MEDV FROM PREDICT (MODEL =  @model, DATA = predict_input) WITH (variable1 float) AS p

(506 rows affected)

Total execution time: 00:00:00.146

id,MEDV
1,23.0990371704102
2,25.439245223999
3,25.439245223999
4,25.8123207092285
5,25.8123207092285
6,25.8123207092285
7,23.5738620758057
8,23.5738620758057
9,23.5738620758057
10,23.5738620758057


Now, you can see the model's predicted median house value for the first thousand samples. 